In [ ]:
import os
from osgeo import gdal,ogr,osr
from tqdm import tqdm
import re
import pickle

In [ ]:
def calc_map_LonLat(nlcd_path):
    def intermidiate_calc(extracted_xy):
        ms  = re.search('\..*"',  extracted_xy).group()[1:-1]
        sec = re.search('\'.*\.', extracted_xy).group()[1:-1]
        min = re.search('d.*\'',  extracted_xy).group()[1:-1]
        deg = re.search('.*d',    extracted_xy).group()[0:-1]
        return(int(deg) + int(min) / 60 + int(sec) / 3600 + int(ms) / 3600000)
    
    def calc_mapXY(extracted_UL_LR):
        mapX = - intermidiate_calc(re.search('\).*W', extracted_UL_LR).group()[3:-1])
        mapY =   intermidiate_calc(re.search('W,.*N', extracted_UL_LR).group()[2:-1])
        return(mapX, mapY)

    txt = gdal.Info(nlcd_path)
    extracted_UL = re.search('Upper Left.*\nLower Left', txt).group()
    extracted_LR = re.search('Lower Right.*\nCenter', txt).group()
    _mapXmin_, _mapYmax_ = calc_mapXY(extracted_UL)
    _mapXmax_, _mapYmin_ = calc_mapXY(extracted_LR)

    return(_mapXmin_,_mapYmin_, _mapXmax_, _mapYmax_)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/training_set_naip_nlcd_both.csv')
df = df[df.group == 0] 

In [ ]:
os.makedirs('./data', exist_ok=True)

!rm -rf ./data/label/*

df['label_fn'].to_csv('./data/label_fns.txt', header=False, index=False)

!wget --quiet -P ./data/label -i ./data/label_fns.txt

In [ ]:
coors = []
for file in tqdm(sorted([file for file in os.listdir('./data/label/') if file.endswith('.tif')])):
    f_path = os.path.join('./data/label/', file)
    ds = gdal.Open(f_path, gdal.GA_ReadOnly)

    _mapXmin_,_mapYmin_, _mapXmax_, _mapYmax_ = calc_map_LonLat(f_path)

    xdim = ds.RasterXSize
    ydim = ds.RasterYSize
    
    if xdim % 10 == 0:
        dw_xdim = xdim // 10
    else:
        dw_xdim = xdim
    
    if ydim % 10 == 0:
        dw_ydim = ydim // 10
    else:
        dw_ydim = ydim        

    dw_name = re.search('\d+_',  file).group()[:-1]
    
    coorset = [dw_name, xdim, ydim, dw_xdim, dw_ydim, [_mapXmin_,_mapYmin_, _mapXmax_, _mapYmax_]]
    coors.append(coorset)

100%|██████████| 2250/2250 [00:11<00:00, 195.65it/s]


In [ ]:
f = open('./drive/MyDrive/coor.txt', 'wb')
pickle.dump(coors, f)